In [ ]:
# install and import lib
!pip install keras_tuner
import tensorflow as tf
from tensorflow.keras import layers, callbacks
import keras_tuner as kt
import datetime
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.2 MB/s eta 0:00:00


In [ ]:
# load and prepare dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# define model builder for keras tuner
def build_model(hp):
    units = hp.Choice("dense_units", values=[64, 128, 256])
    dropout_rate = hp.Float("dropout_rate", min_value=0.2, max_value=0.5, step=0.1)

    model = tf.keras.Sequential([
        layers.Dense(units, activation='relu', input_shape=(784,)),
        layers.Dropout(dropout_rate),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
# setup tensorboard logging
log_dir = os.path.join("logs", "fit", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tb_callback = callbacks.TensorBoard(log_dir=log_dir)

In [ ]:
# add EarlyStopping
early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

In [ ]:
# create a tuner
tuner = kt.Hyperband(
    build_model,
    objective="val_accuracy",
    max_epochs=10,
    factor=3,
    directory="tuner_logs",
    project_name="fashionmnist_opt"
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# search for the best model
tuner.search(x_train, y_train,
             validation_split=0.2,
             epochs=15,
             batch_size=128,
             callbacks=[tb_callback, early_stop],
             verbose=1)

Trial 9 Complete [00h 00m 09s]
val_accuracy: 0.8475000262260437

Best val_accuracy So Far: 0.8621666431427002
Total elapsed time: 00h 01m 15s


In [ ]:
# retrieve best model and evaluate
best_model = tuner.get_best_models(num_models=1)[0]
loss, acc = best_model.evaluate(x_test, y_test, verbose=0)
print(f"✅ Best model test accuracy: {acc:.4f}")

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


✅ Best model test accuracy: 0.8529
